In [1]:
%%html
<style>
  table { float: left; background: #d9d9d9; }
</style>

# find_objメソッド

```python
def find_obj(content, vimtype, name, first=True)
```

vimtypeを指定してManaged Objectを検索するメソッドです。  

## 引数

| args    | type                     | param                                          |
| :-      | :----------------------- | :--------------------------------              |
| content | class                    | ServiceContent Object                          |
| vimtype | class                    | 検索対象のオブジェクトタイプを指定             |
| name    | str                      | 検索するオブジェクト名を指定                   |
| first   | bool                     | 最初にnameにマッチしたオブジェクトを返すフラグ |

## 戻り値

| rtype             | return                                                                                      |
| :---------------- | :------------------------------------------------------------------------------------------ |
| class             | firstがTrueの時、nameにマッチしたManaged Objectを返す。もし、マッチしない場合はNoneを返す   |
| list              | firstがFalseの時listで結果を返す                                                            |

# find_objメソッド動作確認

find_objメソッドの動作確認をしてみます。  
以下の変数は環境に合わせて変更してください。

* vm_name
* hostname
* username
* password
* validate_certs

戻り値はfirstがTrueの場合とFalseで違いを確認してみます。

In [ ]:
from ansible.module_utils.vmware import connect_to_api, find_obj
from pyVmomi import vim

vm_name = 'VM Name'

class module():
    def __init__(self):
        self.params = {}

m = module()
m.params['hostname'] = 'vCenter IP or HostName'
m.params['username'] = 'UserName'
m.params['password'] = 'Password'
m.params['validate_certs'] = False

sc = connect_to_api(m)
vm = find_obj(sc, [vim.VirtualMachine], vm_name)
print(vm.name)

vm_list = find_obj(sc, [vim.VirtualMachine], vm_name, False)
print(vm_list)

# find_objメソッドを使ったモジュール例

find_objを使って実際にAnsibleモジュールを作ってみましょう。  
以下のモジュールはVM名をしてフィルタリングされた結果を戻すだけの単純なものです。  
モジュールは `/tmp/ansible_salf_made_module` に `vm_name.py` で保存されます。  
`/tmp/ansible_salf_made_module` ディレクトリが無い場合は作成してください。  
既にディレクトリがある場合は、以下を実行してモジュールファイルを作成します。

In [ ]:
%%writefile /tmp/ansible_salf_made_module/vm_name.py
#!/usr/bin/python

try:
    from pyVmomi import vim, vmodl
    HAS_PYVMOMI = True
except ImportError:
    HAS_PYVMOMI = False

from ansible.module_utils.vmware import find_obj, connect_to_api, vmware_argument_spec
from ansible.module_utils.basic import AnsibleModule

def main():
    argument_spec = vmware_argument_spec()
    argument_spec.update(name=dict(type='str', required=True))

    module = AnsibleModule(argument_spec=argument_spec, supports_check_mode=True)

    if not HAS_PYVMOMI:
        module.fail_json(msg='pyvmomi python library not found')

    vm_name = module.params['name']

    result = dict(changed=False)
    sc = connect_to_api(module)
    vm = find_obj(sc, [vim.VirtualMachine], vm_name)
    if(vm):
        result['vm'] = vm.name
        module.exit_json(**result)
    else:
        module.fail_json(msg="%s not found" % vm_name)

if __name__ == "__main__":
    main()

# モジュールを動かす

作ったモジュールを実際に動かしてみます。  
以下のオプションは環境に合わせて変更してください。

* hostname
* username
* password
* validate_certs
* name

変更後、Kernelを `Ansible` に変更して以下を実行します。

In [ ]:
#ansible.cfg
[defaults]
library = /tmp/ansible_salf_made_module

In [ ]:
#inventory
[all]
localhost

In [ ]:
#play
name: salf-made module test
hosts: localhost
connection: local
gather_facts: no

In [ ]:
#task
vm_name:
  hostname: vCenter IP or HostName
  username: UserName
  password: Password
  validate_certs: no
  name: VM Name

`name` に指定したVM名が取得できていることを確認します。